# Дерево Фенвика

- Структура, позволяющая на многих задачах заменить собой дерево отрезков.
- Работает намного быстрее (на сумме в ~5 раз быстрее ДО с e-maxx.ru).
- Пишется быстрее и проще -- трудно запомнить только две формулы.
- Очень легко обобщается на б*о*льшие размерности, дружит с хэш-таблицами и другими внутренними структурами
- Занимает памяти **ровно** столько же, сколько бы потребовалось на хранение такого же массива.

 Пусть дан массив $a$ длины $n$. Деревом фенвика будем называть массив $t$ той же длины, который объявим так: 
 
 $$ t_i = \sum_{k=F(i)}^i a_k $$
 
 Для $F$ выполнено $F(i) \leq i$. Конкретно её определим потом.
 
 Когда нам нужна сумма на отрезке, мы сводим этот запрос к двум суммам на префиксе, каждый из которых будем считать, пользуясь следующим фактом:
 
 $$ sum(k) = t_k + sum(F(k)-1) $$
 
 Когда мы изменяем $k$-ю ячейку исходного массива, нам нужно обновить все $t_i$, в которых учтена эта ячейка.
 
 $F$ можно выбрать так, что обе операции будут работать за $O(\log n)$.

Выберем F так: `F(x) = x & (x+1)`.
и обратная `x = x | (x+1)`. Эта функция удаляет / восстанавливает все последние единицы из x. 

Итак, выберем `F(x) = x & (-x)`. В C++ это будет какая-то степень двойки. Чтобы понять это, вспомним как в компах работают отрицательные числа. Чтобы процессор нигде не ифал знак, он просто считает, что все происходит по модулю $2^k$, где $k$ зависит от операционной системы и типа данных). Чтобы посмотреть, как выглядит отрицательное число, нужно взять его бинарное представление и вычесть из нуля (или из $2^k$ и изменить первый бит).

In [14]:
x = 42

x & (-x)

2

In [11]:
def binary(x):
    print("{0:b}".format(x))
    
x = 42
binary(x)
binary(64-x)
binary(x & (-x))

1101010
1010110
1000010


Чтобы понять, почему всё это называется деревом, нужно немного воображения.

Примерная реализация. `While`-ы при желании можно заменить `for`-ами.

```c++
const int n = 1e5;
int t[maxn];

int sum (int r) {
    int ans = 0;
    while (r >= 0) {
        ans += t[r];
        r = (r & (r+1)) - 1;
    }
    return ans;
}

int sum (int l, int r) {
    return sum(r) - sum(l-1);
}

void add (int k, int x) {
    while (k < n) {
        [k] += x;
        k = k | (k+1);
    }
}
```

# Ограничения

Дерево Фенвика можно использовать, когда наша операция обратима, а также когда трюк с префиксными суммами работает. Это обычно простые операции типа суммы, xor, умножения по модулю (если гарантируется, что на этот модуль ничего не делится). Минимум и gcd, отложенные операции и персистентность прикрутить в общем случае уже не получится -- тогда уже нужно писать ДО.

# 2d, 3d...

> $k$-мерное дерево Фенвика пишется в $(k+1)$ строчку

Тут уже явно удобнее использовать `for`, чем `while`. Нужно добавить одну такую же строчку в `sum`, `add`, а также вместо двух запросов к префиксным суммам использовать четыре.

```c++
int sum (int r1, int r2) {
    int ans = 0;
    for (int i = r1; i >= 0; i = (i&(i+1))-1)
        for (int j = r2; j >= 0; j = (j&(j+1))-1)
            ans += t[i][j];
    return ans;
}
```

Вот моё решение какой-то задачи на 2d сумму с USACO 2017: https://pastebin.com/DPemaJeW